UR Clustering - scratch pad

In [ ]:
%reset -f

from Bio import SeqIO
from Bio.Cluster import kcluster


file = 'ur.fasta'
ur = [record.id for record in SeqIO.parse(file, 'fasta')]
print(ur)

# clustering


In [14]:
%%latex

This is a latex cell. You can define $\lambda_i$, for all $i\in\{0,\dots,n\}$ and
\[
a = \frac{m}{k}.    
\]
and
\begin{equation}
\dot{x} = q_{i-1}(x) - q_i(x), \quad \text{for all } i=1,\dots,n.
\label{eq:sample}
\end{equation}

<IPython.core.display.Latex object>

# General Purpose Functions

In [ ]:
# THIS WAS COPIED INTO THE FILE mysequtils.py !!

from Bio import SeqIO
def my_fasta_read(fname):
    '''Reads fasta file. 
    The functions returns a list of the sequences, a list of the sequences IDs 
    and a list of the sequences descriptions (content of the text following >>)'''
    seq = []
    iD = []
    desc = []  # description 
    for record in SeqIO.parse(fname, "fasta"):
        desc.append(record.description)
        seq.append(str(record.seq))
        iD.append(record.id)
    return seq, iD, desc

## Functions for nucleotide clustering

In [93]:
#%%writefile myseqclusters.py
import itertools as itr
import csv

def NT_base_seq_score(two_seqs, eql=+5, neql=-4):
    '''Returns the score of the distance between two sequences. Default scorings are nuc44-based.'''
    return(sum([eql if s1==s2 else neql for s1,s2 in zip(two_seqs[0], two_seqs[1])]))


def NT_pair_seqs_score(all_seqs, score_func, eql=+5, neql=-4):
    '''Returns a dictionary where the keys are all possible unique pairs of all_seqs, and the values are
    the corresponding scores based on score_func'''
    scores = {}
    for a in itr.combinations(range(len(all_seqs)), 2):
        #scores[all_seqs[a[0]]+'~'+all_seqs[a[1]]] = score_func((all_seqs[a[0]], all_seqs[a[1]]), eql, neql)
        scores[(all_seqs[a[0]], all_seqs[a[1]])] = score_func((all_seqs[a[0]], all_seqs[a[1]]), eql, neql)
    return scores


def parse_starcode_cls_out(file):
    '''This function parses the output of the tool starcode that 
    is used to cluster nucleotide sequences. The returned dictionary contains
    a centroid as a key and a list of the correponding sequences in that
    cluster as a value.'''
    with open(file, 'rt') as fin:
        clusts = [r[0].split('\t') for r in csv.reader(fin, delimiter=' ')]

    clusters = {}
    for info in clusts: clusters[info[0]] = info[-1].split(',')
    return clusters

def parse_meshclsut_cls_out(file):
    '''This function parses the output of the tool Meshclust that 
    is used to cluster nucleotide sequences. The returned dictionary contains
    a centroid as a key and a list of the correponding sequences in that
    cluster as a value.'''
    clusters = {}
    with open(file, 'rt') as fin:
        for line in fin:
            vals = line.split(' ')
            if vals[0]=='>Cluster': # a new cluster
                clst_num = int(vals[-1][0])
                if clst_num > 0: # save the previous cluster
                    clusters[centroid]=clst_seqs
                clst_seqs = []
                centroid = None
            else:
                seq = re.search('(\w+)', vals[2]).group(0)
                clst_seqs.append(seq)
                if(vals[-1][0]=='*'):
                    centroid=seq
        # save the last cluster
        clusters[centroid]=clst_seqs
    return clusters


Overwriting myseqclusters.py


In [ ]:
'''Testing the functions above'''

from pprint import pprint
import mysequtils as myut

#A = ('CTGC', 'ATGG')
#print('base score of {}:{} is {}'.format(A[0], A[1], NT_base_seq_score(A)))


import itertools as itr
#all_seqs = ('ATGC', 'ATTG', 'ACGT', 'ATGG')
#all_scores=NT_pair_seqs_score(all_seqs, NT_base_seq_score)
#pprint(all_scores)


from Bio import SeqIO
ur_file = 'ur.fasta'
ur1_file = 'ur1.fasta'

seq, iD, desc = my_fasta_read(ur1_file)
#for i, s, d in zip(iD, seq, desc):
#    print('id:{}, desc:{}, seq:{}'.format(i,d, s))





#urs = [x.id for x in list(SeqIO.parse(ur_file, "fasta"))]
urs,_,_ = my_fasta_read(ur_file)
#print(urs)
#urs_scores=NT_pair_seqs_score(urs, NT_base_seq_score, 1, -1)




# this is the UR set of vtaxid=1090134
myur = { 'TCA', 'TGT', 'AGT', 'TTT', 'GAG', 'CAA', 'CTC', 'GAC' }
urs_scores=NT_pair_seqs_score(list(myur), NT_base_seq_score, 1, -1)
#pprint(urs_scores)

from operator import itemgetter
aa = sorted(urs_scores.items(), key=itemgetter(1), reverse=True)
#print(aa)
bb = sorted(aa, key=lambda entry: entry[0][0])
#bb = sorted(urs_scores.items(), key=lambda entry: entry[0][0])

for a in bb:
    print('{}:{}'.format(a[0], a[1]))
    
# saving ur in a fasta file
fasta_name = './ur2.fasta'
with open(fasta_name, 'wt') as fout:
    print(myut.seq_create_fasta(myur,myur), file=fout)
print('file saved in ', fasta_name)


# this is the UR set of vtaxid=1605721
myur3 = {'GGAG', 'GCTT', 'AAGC', 'TCAG', 'CGGA', 'CTGA', 'AACG', 'GTTC', \
 'ACGT', 'AGCT', 'TCCG', 'AGCC', 'CTTC', 'GTTA', 'GGGG', 'GAAG', \
 'CGTT', 'CTCC', 'CCCC', 'CTTA', 'TTCG', 'GGAA', 'GGGA'}
# saving ur in a fasta file
fasta_name = './ur3.fasta'
with open(fasta_name, 'wt') as fout:
    print(myut.seq_create_fasta(myur3,myur3), file=fout)
print('file saved in ', fasta_name)


# Parse the output of starcode

In [ ]:
import csv
from pprint import pprint

infile = 'ur2.fasta'
file = 'ur_clust'  # startcode output

ur_seq, _, _ = my_fasta_read(infile)

with open(file, 'rt') as fin:
    cin = csv.reader(fin, delimiter=' ')
    clusts = [r for r in cin]

print('urs = {}'.format(ur_seq))
print('Found {} clusters:'.format(len(clusts)))
for x in clusts:
    info = x[0].split('\t')
    #print('centroid={}, members={}'.format(info[0], info[-1]))
    print('{}:{}'.format(info[0], info[-1].split(',')))

    
clusters = parse_starcode_cls_out(file)
pprint(clusters)


file3 = 'ur3_clust'  # startcode output
clusters3 = parse_starcode_cls_out(file3)
pprint(clusters3)









## Running Starcode. 

This is an example of clustering a list of UR sequences using starcode.

In [19]:
from pprint import pprint
import mysequtils as myut
import subprocess


str_exe = '~/work/mystuff/tools/starcode/starcode'
#str_flags = ' -r 1 --print-clusters'   # message-passing algorithm
str_flags = ' -s --print-clusters'   # sphere algorithm

base_command = str_exe + str_flags

# vtaxid = 458639
myur4 = {'TTTC', 'CCAG', 'TTTT', 'AATT', 'GCCA', 'CCCA', \
         'AAAT', 'GATC', 'CGCG', 'GCGC', 'AAAA', 'GTCT', 'TCAG'}

# vtaxid = 445686
#myur4 = {'GGCT', 'AGCC', 'GCTA', 'GCTT', 'TTTT', 'AATT', \
#         'AAAT', 'CGCG', 'AAAA', 'GTCC', 'GGCC', 'CCGG', 'GTAC'}
#infile = 'ur4.fasta'
infile = 'ur4.seq'
outfile = 'ur4_clust'


command =  base_command + ' -i ' + infile  + ' -o ' + outfile

# generate a fasta file containing the 
#with open(infile, 'wt') as fout:
#    print(myut.seq_create_fasta(myur4,myur4), file=fout)
# generate a file containing the sequences (a sequence per line)
myut.seq2plain_text_file(myur4, infile)
    
# run starcode
print('running: ', command)
ret, stdout = subprocess.getstatusoutput(command) #os.system(command)

if ret == 0:
    # parse starcode output
    clusters = parse_starcode_cls_out(outfile)
    pprint(clusters)
    print('Found {} clusters:'.format(len(clusters)))
    for k in clusters.keys(): print(clusters[k])
    
else:
    print('starcode errored and returned: ', stdout)



running:  ~/work/mystuff/tools/starcode/starcode -s --print-clusters -i ur4.seq -o ur4_clust
{'AATT': ['AATT', 'AAAT', 'AAAA', 'GATC'],
 'CGCG': ['CGCG'],
 'GCCA': ['GCCA', 'CCCA', 'GCGC', 'GTCT'],
 'TCAG': ['TCAG', 'CCAG'],
 'TTTC': ['TTTC', 'TTTT']}
Found 5 clusters:
['AATT', 'AAAT', 'AAAA', 'GATC']
['GCCA', 'CCCA', 'GCGC', 'GTCT']
['TCAG', 'CCAG']
['TTTC', 'TTTT']
['CGCG']


scratch pad

In [27]:
from collections import OrderedDict

aa = OrderedDict()

aa[('a1', 11)] = 21
aa[('a2', 12)] = 22
aa[('a3', 13)] = 23

for k in aa.keys(): print('{}:{}'.format(k, aa[k]))
    
    
'_'.join(['a', 'b', 'v'])

('a1', 11):21
('a2', 12):22
('a3', 13):23


'a_b_v'

## Using Meshclust to cluster

In [90]:
import re
from pprint import pprint


# starcode parameters (we use Starcode to cluster)
mesh_exe = '~/work/mystuff/tools/MeShClust-master/src/cluster/meshclust'
mesh_flags = '--id 0.5'
# ==========================================================================
infile = 'ur.fasta'
outfile = 'clst.mesh'

mesh_flags = ' '.join([infile, mesh_flags, '--output '+outfile])
command = mesh_exe + ' ' + mesh_flags
#print(mesh_flags)
ret, sout = subprocess.getstatusoutput(command)
#print(ret, sout)

# parse the output
clusters = {}
with open(outfile, 'rt') as fin:
    for line in fin:
        #print(line)
        vals = line.split(' ')
        #print(vals)
        if vals[0]=='>Cluster':
            clst_num = int(vals[-1][0])
            if clst_num > 0: # save the previous cluster
                clusters[centroid]=clst_seqs
            # new cluster (ends previous cluster)
            #print('A new cluster #{}'.format(clst_num))
            clst_seqs = []
            centroid = None
        else:
            seq = re.search('(\w+)', vals[2]).group(0)
            clst_seqs.append(seq)
            #print('\t',seq)
            if(vals[-1][0]=='*'):
                #print('\tthis is the centroid')
                centroid=seq
    # save the last cluster
    clusters[centroid]=clst_seqs

print('Found {} clusters:'.format(len(clusters)))
for k,v in clusters.items(): print('{}:{}'.format(k,v))
    
    
clusters_f = parse_meshclsut_cls_out(outfile)    
print('\nNow using the function:')
print('Found {} clusters:'.format(len(clusters_f)))
for k,v in clusters_f.items(): print('{}:{}'.format(k,v))


Found 5 clusters:
AATTG:['AATTG', 'ATTTT', 'AAATT', 'ACGTG', 'AATTC', 'TTTTT', 'AGCTT', 'TTTTC', 'TTTCA']
AGCCA:['AGCCA', 'AGGCT', 'AGCCT', 'AGCTC', 'AGGCC', 'GGCTA', 'AGCTA', 'GGCTT', 'TGGCT', 'AAGCT', 'GAGCT', 'CAGCT', 'AAAAT', 'GGCTG', 'GGCTC']
ACCGG:['ACCGG', 'GCTGG']
AAAAA:['AAAAA']
CAATT:['CAATT']

Now using the function:
Found 5 clusters:
AATTG:['AATTG', 'ATTTT', 'AAATT', 'ACGTG', 'AATTC', 'TTTTT', 'AGCTT', 'TTTTC', 'TTTCA']
AGCCA:['AGCCA', 'AGGCT', 'AGCCT', 'AGCTC', 'AGGCC', 'GGCTA', 'AGCTA', 'GGCTT', 'TGGCT', 'AAGCT', 'GAGCT', 'CAGCT', 'AAAAT', 'GGCTG', 'GGCTC']
ACCGG:['ACCGG', 'GCTGG']
AAAAA:['AAAAA']
CAATT:['CAATT']
